# SDES-2018 Building stock - CONFIDENTIAL

In [1]:
import os
import pandas as pd
from IPython.display import display

In [2]:
def reindex_mi(df, mi_index, levels=None, axis=0):
    """Return re-indexed DataFrame based on miindex using only few labels.

    Parameters
    -----------
    df: pd.DataFrame, pd.Series
        data to reindex
    mi_index: pd.MultiIndex, pd.Index
        master to index to reindex df
    levels: list, default df.index.names
        list of levels to use to reindex df
    axis: {0, 1}, default 0
        axis to reindex df

    Returns
    --------
    pd.DataFrame, pd.Series

    Example
    -------
        reindex_mi(surface_ds, segments, ['Occupancy status', 'Housing type']))
        reindex_mi(cost_invest_ds, segments, ['Heating energy final', 'Heating energy']))
    """

    if isinstance(df, (float, int)):
        return pd.Series(df, index=mi_index)

    if levels is None:
        if axis == 0:
            levels = df.index.names
        else:
            levels = df.columns.names

    if len(levels) > 1:
        tuple_index = (mi_index.get_level_values(level).tolist() for level in levels)
        new_miindex = pd.MultiIndex.from_tuples(list(zip(*tuple_index)))
        if axis == 0:
            df = df.reorder_levels(levels)
        else:
            df = df.reorder_levels(levels, axis=1)
    else:
        new_miindex = mi_index.get_level_values(levels[0])
    df_reindex = df.reindex(new_miindex, axis=axis)
    if axis == 0:
        df_reindex.index = mi_index
    elif axis == 1:
        df_reindex.columns = mi_index
    else:
        raise AttributeError('Axis can only be 0 or 1')

    return df_reindex


In [3]:
folder_input = 'sdes_2018'
folder_output = 'output'
if not os.path.isdir(folder_output):
    os.mkdir(folder_output)

# Main input
Sources: SDES-2018
Status: Confidential

In [4]:
replace_dict = {r'^(P)$': 'Owner-occupied',
                'Gaz': 'Natural gas',
                'Bois\*': 'Wood fuel',
                'Fioul domestique': 'Oil fuel',
                'Gaz de pétrole liquéfié': 'Oil fuel',
                'Chauffage urbain': 'Heating',
                'MA': 'Single-family',
                'Maison': 'Single-family',
                'AP': 'Multi-family',
                'Appartement': 'Multi-family',
                'LP': 'Privately rented',
                'Autres.*': 'Others',
                'LS': 'Social-housing',
                '.?lectricit.*': 'Electricity',}

## Cleaning names

In [7]:
name_file = 'comptages_DPE.csv'
stock_buildings = pd.read_csv(os.path.join(folder_input, name_file), sep=',', header=[0], encoding='latin-1',
                        index_col=[0, 1, 2, 3, 4]).squeeze()
index_names = ['Housing type', 'Occupancy status', 'Income tenant', 'Heating energy', 'Energy performance']
stock_buildings.index.set_names(index_names, inplace=True)

stock_buildings = stock_buildings.reset_index().replace(replace_dict, regex=True).set_index(stock_buildings.index.names).squeeze()

display(stock_buildings.head().to_frame().style.format('{:.1f}'))
display(stock_buildings.groupby('Heating energy').sum() / 10**6)
print('Total number of housing in this study {:,.0f}'.format(stock_buildings.sum()))

,,,,,NB_LOG
Housing type,Occupancy status,Income tenant,Heating energy,Energy performance,
Multi-family,G,D1,Others,A,3.4
Single-family,G,D1,Others,A,20.0
Multi-family,Privately rented,D1,Others,A,282.2
Single-family,Privately rented,D1,Others,A,343.9
Multi-family,Social-housing,D1,Others,A,887.9


Heating energy
Electricity    10.456509
Natural gas    11.725657
Others          6.438182
Name: NB_LOG, dtype: float64

Total number of housing in this study 28,620,348


In [8]:
stock_buildings.groupby('Energy performance').sum() / 10**6

Energy performance
A    0.519791
B    1.370423
C    5.161098
D    9.791445
E    6.992731
F    3.076042
G    1.708818
Name: NB_LOG, dtype: float64

## Remove gratuity

In [9]:
stock_buildings = stock_buildings.loc[stock_buildings.index.get_level_values('Occupancy status') != 'G']
display(stock_buildings.head().to_frame().style.format('{:.1f}'))
print('Total number of housing at this point {:,.0f}'.format(stock_buildings.sum()))

,,,,,NB_LOG
Housing type,Occupancy status,Income tenant,Heating energy,Energy performance,
Multi-family,Privately rented,D1,Others,A,282.2
Single-family,Privately rented,D1,Others,A,343.9
Multi-family,Social-housing,D1,Others,A,887.9
Single-family,Social-housing,D1,Others,A,73.4
Multi-family,Owner-occupied,D1,Others,A,155.4


Total number of housing at this point 28,366,454


In [10]:
stock_buildings.groupby('Energy performance').sum() / 10**6


Energy performance
A    0.514970
B    1.357447
C    5.105653
D    9.698378
E    6.933754
F    3.053258
G    1.702996
Name: NB_LOG, dtype: float64

# Add owner income as attribute for each building

Using another source of data, we add another level (or attribute) to building stocks: income owner.
Income owner is useful to determine socio-economic parameters like the interest rate or the investment duration.

In [19]:
display(stock_buildings.groupby('Occupancy status').sum())

Occupancy status
Owner-occupied      1.638218e+07
Privately rented    7.331687e+06
Social-housing      4.652589e+06
Name: NB_LOG, dtype: float64

## Read data income landlord

In [20]:
name_file = 'parclocatifprive_post48_revenusPB.csv'
data_income_owner = pd.read_csv(os.path.join(folder_input, name_file), sep=',', header=[0],
                                index_col=[2, 0, 3, 5, 6])
display(data_income_owner.head())

,,,,,DATECONST,DECILE_PB,NB_LOG
NATLOC,OCC,DECILE,ENERGIE,ETIQUETTE,,,
AP,LP,D1,Autres,A,Après 1948,D1,1.474775
MA,LP,D1,Autres,A,Après 1948,D1,9.673954
AP,LP,D2,Autres,A,Après 1948,D1,0.952759
MA,LP,D2,Autres,A,Après 1948,D1,9.703615
AP,LP,D3,Autres,A,Après 1948,D1,0.334222


## Cleaning

In [21]:
index_names = ['Housing type', 'Occupancy status', 'Income tenant', 'Heating energy', 'Energy performance']
data_income_owner.index.set_names(index_names, inplace=True)

data_income_owner.rename(columns={'DECILE_PB': 'Income owner'}, inplace=True)
data_income_owner.reset_index(inplace=True)
data_income_owner.set_index(index_names + ['Income owner'], inplace=True)

data_income_owner = data_income_owner.reset_index().replace(replace_dict, regex=True).set_index(data_income_owner.index.names).squeeze()

data_income_owner = data_income_owner.loc[data_income_owner.index.get_level_values('Income owner') != 'NC']
data_income_owner = data_income_owner.loc[:, 'NB_LOG']

display(data_income_owner.head())
print('\n Total number of housing at this point {:,.0f} - stock with income owner'.format(data_income_owner.sum()))

Housing type   Occupancy status  Income tenant  Heating energy  Energy performance  Income owner
Multi-family   Privately rented  D1             Others          A                   D1              1.474775
Single-family  Privately rented  D1             Others          A                   D1              9.673954
Multi-family   Privately rented  D2             Others          A                   D1              0.952759
Single-family  Privately rented  D2             Others          A                   D1              9.703615
Multi-family   Privately rented  D3             Others          A                   D1              0.334222
Name: NB_LOG, dtype: float64


 Total number of housing at this point 2,614,910 - stock with income owner


## Merging data

In [22]:
print('\n Total number of housing at this point {:,.0f}'.format(stock_buildings.sum()))
# multiplication will remove other value than Landlords (need to be added back later)
share_income_owner = (data_income_owner.unstack('Income owner').T / data_income_owner.unstack('Income owner').sum(axis=1)).T
stock_buildings_landlords = (stock_buildings * reindex_mi(share_income_owner, stock_buildings.index).T).T
stock_buildings_landlords = stock_buildings_landlords.stack().squeeze()

stock_buildings_owners = stock_buildings[stock_buildings.index.get_level_values('Occupancy status') == 'Owner-occupied']
stock_buildings_owners = pd.concat((stock_buildings_owners, pd.Series(stock_buildings_owners.index.get_level_values('Income tenant'), index=stock_buildings_owners.index, name='Income owner')), axis=1)
stock_buildings_owners = stock_buildings_owners.set_index('Income owner', append=True).squeeze()

stock_buildings_social = stock_buildings[stock_buildings.index.get_level_values('Occupancy status') == 'Social-housing']
stock_buildings_social = pd.concat((stock_buildings_social, pd.Series('D10', index=stock_buildings_social.index, name='Income owner')), axis=1)
stock_buildings_social = stock_buildings_social.set_index('Income owner', append=True).squeeze()

stock_buildings = pd.concat((stock_buildings_landlords, stock_buildings_owners, stock_buildings_social))

print(stock_buildings)
print('\n Total number of housing at this point {:,.0f}'.format(stock_buildings.sum()))


 Total number of housing at this point 28,366,454
Housing type   Occupancy status  Income tenant  Heating energy  Energy performance  Income owner
Multi-family   Privately rented  D1             Others          A                   D1               16.845584
                                                                                    D10             110.377148
                                                                                    D2                5.923017
                                                                                    D3               12.285413
                                                                                    D4                7.233046
                                                                                                       ...    
Single-family  Social-housing    D8             Natural gas     G                   D10              44.677646
Multi-family   Social-housing    D9             Natural gas     G          

# De-aggregate 'Others' to 'Wood fuel', 'Oil fuel' and 'District heating'

Using another source of data, we de-aggregate each rows where Heating energy == 'Others' to 'Wood fuel' and 'Oil fuel'.  
Rate depends on Housing type

## Read data oil fuel and wood fuel

In [23]:
name_file = 'fuel_oil_wood_2018.xlsx'
data_fuel = pd.read_excel(os.path.join(folder_input, name_file), header=[0], index_col=[1, 0])
display(data_fuel.head(10))

data_fuel.index.set_names(['Heating energy', 'Housing type'], inplace=True)
data_fuel = data_fuel.reset_index().replace(replace_dict, regex=True).set_index(data_fuel.index.names).squeeze()

fuel_list = ['Wood fuel', 'Oil fuel', 'Heating']
data_fuel = data_fuel.loc[data_fuel.index.get_level_values('Heating energy').isin(fuel_list), 'Taux du parc en %']
display(data_fuel.head(10).to_frame().style.format('{:.2f}'))

,,Parc en milliers de logements,Taux du parc en %
Energie principale de chauffage,Type de logement,,
Autres énergies,Appartement,19.155201,0.010077
Bois*,Appartement,51.225000,0.026947
Chauffage urbain,Appartement,1234.451978,0.649395
Fioul domestique,Appartement,585.193629,0.307847
Gaz de pétrole liquéfié,Appartement,10.900000,0.005734
Autres énergies,Maison,168.656387,0.036464
Bois*,Maison,1197.667239,0.258939
Fioul domestique,Maison,2867.700000,0.620005
Gaz de pétrole liquéfié,Maison,391.261813,0.084592


,,Taux du parc en %
Heating energy,Housing type,
Wood fuel,Multi-family,0.03
Heating,Multi-family,0.65
Oil fuel,Multi-family,0.31
Wood fuel,Single-family,0.26
Oil fuel,Single-family,0.62


In [24]:
## Merging data_fuel with stock_buildings
print('\n Total number of housing at this point {:,.0f}'.format(stock_buildings.sum()))
print(stock_buildings.groupby('Heating energy').sum())

data_fuel = data_fuel.to_frame().pivot_table(columns='Heating energy', index='Housing type').droplevel(None, axis=1)
data_fuel = pd.concat([data_fuel], keys=['Others'], names=['Heating energy'], axis=0)
display(data_fuel.style.format('{:.0%}'))

data_fuel = reindex_mi(data_fuel, stock_buildings.index)

# multiplication will remove other value than Others (need to be added back later)
stock_buildings_others = (stock_buildings * data_fuel.T).T
stock_buildings_others = stock_buildings_others.droplevel('Heating energy', axis=0).stack().squeeze()
stock_buildings_others.dropna(inplace=True)
stock_buildings_others = stock_buildings_others.reorder_levels(stock_buildings.index.names)

stock_buildings = pd.concat((stock_buildings.loc[stock_buildings.index.get_level_values('Heating energy') != 'Others'], stock_buildings_others), axis=0)
display(stock_buildings.groupby('Heating energy').sum())
print('\n Total number of housing at this point {:,.0f}'.format(stock_buildings.sum()))



 Total number of housing at this point 28,366,454
Heating energy
Electricity    1.036755e+07
Natural gas    1.161841e+07
Others         6.380499e+06
dtype: float64


Heating energy
Electricity    1.036755e+07
Heating        1.068541e+06
Natural gas    1.161841e+07
Oil fuel       3.442304e+06
Wood fuel      1.270432e+06
dtype: float64


 Total number of housing at this point 27,767,231


In [11]:
stock_buildings.groupby('Energy performance').sum()

Energy performance
A    5.149695e+05
B    1.357447e+06
C    5.105653e+06
D    9.698378e+06
E    6.933754e+06
F    3.053258e+06
G    1.702996e+06
Name: NB_LOG, dtype: float64

# Export results

In [12]:
print(stock_buildings.to_frame())
print('\n Total number of housings {:,.0f}'.format(stock_buildings.sum()))
stock_buildings.to_csv(os.path.join(folder_output, 'building_stock_sdes2018_aggregated.csv'))

                                                                                     NB_LOG
Housing type  Occupancy status Income tenant Heating energy Energy performance             
Multi-family  Privately rented D1            Others         A                    282.158753
Single-family Privately rented D1            Others         A                    343.899401
Multi-family  Social-housing   D1            Others         A                    887.854956
Single-family Social-housing   D1            Others         A                     73.430425
Multi-family  Owner-occupied   D1            Others         A                    155.393922
...                                                                                     ...
Single-family Privately rented D10           Natural gas    G                    378.653117
Multi-family  Social-housing   D10           Natural gas    G                    513.336530
Single-family Social-housing   D10           Natural gas    G                   

In [13]:
stock_buildings.groupby(['Housing type', 'Heating energy']).sum()

Housing type   Heating energy
Multi-family   Electricity       4.964876e+06
               Natural gas       6.274891e+06
               Others            1.645440e+06
Single-family  Electricity       5.402671e+06
               Natural gas       5.343517e+06
               Others            4.735059e+06
Name: NB_LOG, dtype: float64

In [14]:
stock_buildings.groupby('Energy performance').sum()

Energy performance
A    5.149695e+05
B    1.357447e+06
C    5.105653e+06
D    9.698378e+06
E    6.933754e+06
F    3.053258e+06
G    1.702996e+06
Name: NB_LOG, dtype: float64